In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import html
import requests
import pandas as pd 


<h1> 뉴스 크롤링 미리보기

In [2]:

a = urlopen("https://kidd.co.kr/search.php?find=%ED%8F%AC%EC%8A%A4%EC%BD%94&all=&term=&writer=&page=1")
# a를 호출해 html을 parser
soup = BeautifulSoup(a.read(), 'html.parser')
# soup에서 h1 tag 하나를 가져온다.

c = soup.find_all('a', class_ = lambda x: x == 'list-news')#.find_all('div',class_ = 'list-news-body')
    
for div in c:
    h3 = div.find('h3')
    day = div.find('span',class_ = 'text-muted fs-6 ml-2 ml-lg-3')
    p = div.find('p')
    href = div.get('href')  # div의 상위에 있는 a 태그 찾기
    
    if h3:
        print("[제목]", h3.get_text(strip=True))
    if day:
        print("[날짜]", day.get_text(strip=True))
    if p:
        print("[요약]", p.get_text(strip=True))
    if href:
        print("[링크]", 'https://kidd.co.kr' + href)

    print("-" * 50)  # 구분선

[제목] 포스코퓨처엠·삼성SDI·SK온의 배터리 시장 위기 돌파 전략은?
[날짜] 2025.03.05
[요약] 글로벌 배터리 전시회인 ‘인터배터리 2025 (INTER BATTERY)’가 5일 삼성동 코엑스(COEX)에서 막을 올렸다. 한국 배터리 전문 기업 삼성SDI·SK온·LG에너지솔루션·포스코퓨처엠·에코프로비엠 등도 전시회에 참가해 차세대 배터리 ..
[링크] https://kidd.co.kr/news/240692
--------------------------------------------------
[제목] “에너지 기업 3사 ‘횡재세’ 부과하고, 가스 수급 공공성 강화해야”
[날짜] 2025.02.25
[요약] “한국가스공사의 누적된 미수금 14조 원 중 주택용 사용에서 발생한 미수금은 공익서비스에 따른 사회적 적자, 즉 ‘착한 적자’로 규정하고 정부의 직접적인 재정 지원을 통해 해결해야 한다”
에너지노동사회네트워크 구준모 기..
[링크] https://kidd.co.kr/news/240584
--------------------------------------------------
[제목] 탄소배출권 부당이익 차단…온실가스 거래법 시행령 개정
[날짜] 2025.02.01
[요약] 온실가스 감축 노력 없이 남는 탄소배출권을 판매해 부당이익을 얻는 사례를 막기 위해 환경부가 온실가스 거래법 시행령을 개정했다. 
환경부는 온실가스 배출권거래 제도의 개선 내용을 담은 ‘온실가스 배출권의 할당 및 거래에 관..
[링크] https://kidd.co.kr/news/240219
--------------------------------------------------
[제목] K-배터리 산업, 대내외 파고를 넘기 위해서는 정책 지원 강화돼야
[날짜] 2025.01.16
[요약] ‘K-배터리’로 통칭되면서 유명세를 얻어 온 한국의 배터리 산업이 위기에 봉착했다는 신호가 감지되고 있다. 이에 급변하는 시장의 흐름을 반영한 정부의 지원이 있어야 K-배터리의 명맥을

<h1> 크롤링 정보 엑셀로 저장

In [3]:
news_data = []

for i in range (1,20):
    
    a = urlopen(f"https://kidd.co.kr/search.php?find=%ED%8F%AC%EC%8A%A4%EC%BD%94&all=&term=&writer=&page={i}")
    # a를 호출해 html을 parser
    soup = BeautifulSoup(a.read(), 'html.parser')
    b = soup.select('a.list-news')  # 이건 기본적으로 포함 여부 기반

    # 정확히 'list-news' 하나만 있는 경우만 필터링
    b = [tag for tag in soup.select('a.list-news') if tag.get('class') == ['list-news']]

    # 정보 추출
    for div in b:
        h3 = div.find('h3')
        day = div.find('span', class_='text-muted fs-6 ml-2 ml-lg-3')
        p = div.find('p')
        href = div.get('href')

        news_data.append({
            '제목': h3.get_text(strip=True) if h3 else '',
            '요약': p.get_text(strip=True) if p else '',
            '날짜': day.get_text(strip=True) if day else '',
            '링크': 'https://kidd.co.kr' + href if href else ''
        })

# pandas DataFrame 생성
df = pd.DataFrame(news_data)

# 엑셀 파일로 저장
df.to_excel('포스코_뉴스_리스트.xlsx', index=False)

df['날짜'] = pd.to_datetime(df['날짜'])
df['날짜'] = df['날짜'].sort_values(ascending=False)

print("엑셀 파일 저장 완료!")

엑셀 파일 저장 완료!


In [4]:
df

,제목,요약,날짜,링크
0,포스코퓨처엠·삼성SDI·SK온의 배터리 시장 위기 돌파 전략은?,글로벌 배터리 전시회인 ‘인터배터리 2025 (INTER BATTERY)’가 5일 ...,2025-03-05,https://kidd.co.kr/news/240692
1,"“에너지 기업 3사 ‘횡재세’ 부과하고, 가스 수급 공공성 강화해야”",“한국가스공사의 누적된 미수금 14조 원 중 주택용 사용에서 발생한 미수금은 공익서...,2025-02-25,https://kidd.co.kr/news/240584
2,탄소배출권 부당이익 차단…온실가스 거래법 시행령 개정,온실가스 감축 노력 없이 남는 탄소배출권을 판매해 부당이익을 얻는 사례를 막기 위해...,2025-02-01,https://kidd.co.kr/news/240219
3,"K-배터리 산업, 대내외 파고를 넘기 위해서는 정책 지원 강화돼야",‘K-배터리’로 통칭되면서 유명세를 얻어 온 한국의 배터리 산업이 위기에 봉착했다는...,2025-01-16,https://kidd.co.kr/news/240039
4,"철강업계, 급변하는 업계 흐름에 뱀의 지혜로 대응하자",철강산업은 우리나라 경제의 근간을 이루는 주요 산업이지만 대표적인 ‘굴뚝산업’인 만...,2025-01-14,https://kidd.co.kr/news/239997
...,...,...,...,...
185,스테인리스 생산·전기자동차(EV) 수요 증가 영향 니켈 가격 ‘급상승’,글로벌 니켈 가격이 전기자동차(EV) 시장 확대 등의 이유로 빠르게 상승하고 있는 ...,2019-12-05,https://kidd.co.kr/news/212445
186,"철강협회 선재협의회, 수요산업 간 상생 발전의 장 마련",올해 상반기 전극용접봉 생산이 약 14만7천626톤으로 전년동기 대비 8.3% 감소...,2019-11-25,https://kidd.co.kr/news/212164
187,"미국, EU 등 선진국, 기술·정보 유출 우려 외국인직접투자(FDI) 규제 확대",국가별 경제성장의 주요 지표 중 하나로 평가받는 외국인직접투자 유입 규모가 최근 선...,2019-11-24,https://kidd.co.kr/news/212153
188,"협동로봇, 산업현장을 바꿀 하나의 도구(Tool)로 봐야",‘협동로봇(Cobot)'이라는 개념이 산업계에 전달이 된 이래 제조현장의 모습은 빠...,2019-11-15,https://kidd.co.kr/news/212012


<h1> 가독성을 위한 엑셀 편집

In [5]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# 데이터프레임 생성 (뉴스 데이터 예시)
df = pd.DataFrame(news_data)

# 1단계: 엑셀 저장
filename = '포스코_뉴스_리스트.xlsx'
df.to_excel(filename, index=False)

# 2단계: openpyxl로 불러오기
wb = load_workbook(filename)
ws = wb.active

# 3단계: 열 너비 자동 조절 + 줄바꿈 설정
for col in ws.columns:
    max_length = 0
    column = col[0].column  # 숫자형 컬럼 번호
    column_letter = get_column_letter(column)  # A, B, C...

    for cell in col:
        # 셀 내용 길이 측정
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
            # 줄바꿈 스타일 적용
            cell.alignment = Alignment(wrap_text=True)

    # 여유 공간 포함한 열 너비 설정
    adjusted_width = max_length + 5
    ws.column_dimensions[column_letter].width = adjusted_width

# 4단계: 저장
wb.save(filename)
print("✅ 엑셀 저장 완료! 셀 너비 + 줄바꿈 적용됐어요.")


✅ 엑셀 저장 완료! 셀 너비 + 줄바꿈 적용됐어요.


In [6]:
df

,제목,요약,날짜,링크
0,포스코퓨처엠·삼성SDI·SK온의 배터리 시장 위기 돌파 전략은?,글로벌 배터리 전시회인 ‘인터배터리 2025 (INTER BATTERY)’가 5일 ...,2025.03.05,https://kidd.co.kr/news/240692
1,"“에너지 기업 3사 ‘횡재세’ 부과하고, 가스 수급 공공성 강화해야”",“한국가스공사의 누적된 미수금 14조 원 중 주택용 사용에서 발생한 미수금은 공익서...,2025.02.25,https://kidd.co.kr/news/240584
2,탄소배출권 부당이익 차단…온실가스 거래법 시행령 개정,온실가스 감축 노력 없이 남는 탄소배출권을 판매해 부당이익을 얻는 사례를 막기 위해...,2025.02.01,https://kidd.co.kr/news/240219
3,"K-배터리 산업, 대내외 파고를 넘기 위해서는 정책 지원 강화돼야",‘K-배터리’로 통칭되면서 유명세를 얻어 온 한국의 배터리 산업이 위기에 봉착했다는...,2025.01.16,https://kidd.co.kr/news/240039
4,"철강업계, 급변하는 업계 흐름에 뱀의 지혜로 대응하자",철강산업은 우리나라 경제의 근간을 이루는 주요 산업이지만 대표적인 ‘굴뚝산업’인 만...,2025.01.14,https://kidd.co.kr/news/239997
...,...,...,...,...
185,스테인리스 생산·전기자동차(EV) 수요 증가 영향 니켈 가격 ‘급상승’,글로벌 니켈 가격이 전기자동차(EV) 시장 확대 등의 이유로 빠르게 상승하고 있는 ...,2019.12.05,https://kidd.co.kr/news/212445
186,"철강협회 선재협의회, 수요산업 간 상생 발전의 장 마련",올해 상반기 전극용접봉 생산이 약 14만7천626톤으로 전년동기 대비 8.3% 감소...,2019.11.25,https://kidd.co.kr/news/212164
187,"미국, EU 등 선진국, 기술·정보 유출 우려 외국인직접투자(FDI) 규제 확대",국가별 경제성장의 주요 지표 중 하나로 평가받는 외국인직접투자 유입 규모가 최근 선...,2019.11.24,https://kidd.co.kr/news/212153
188,"협동로봇, 산업현장을 바꿀 하나의 도구(Tool)로 봐야",‘협동로봇(Cobot)'이라는 개념이 산업계에 전달이 된 이래 제조현장의 모습은 빠...,2019.11.15,https://kidd.co.kr/news/212012


In [8]:
ans = df[df['제목'].str.contains('포스코')]
ans

,제목,요약,날짜,링크
0,포스코퓨처엠·삼성SDI·SK온의 배터리 시장 위기 돌파 전략은?,글로벌 배터리 전시회인 ‘인터배터리 2025 (INTER BATTERY)’가 5일 ...,2025.03.05,https://kidd.co.kr/news/240692
11,"이상민 포스코 부장 “산업 탈탄소화, ‘규제’ 아닌 ‘경제’로 지원해야”",“철강은 제조업의 필수 소재로 국가 경제와 제조업 존립의 기반입니다. 철강 산업의 ...,2024.11.28,https://kidd.co.kr/news/239327
17,"[탄소중립엑스포(EXPO)]포스코홀딩스, ‘하이렉스(HyREX)' 기술 앞세워 탄소...","포스코(POSCO)는 전 세계에서도 손꼽히는 철강 기업으로, 연간 3천700만 톤가...",2024.09.10,https://kidd.co.kr/news/238225
22,"포스코이엔씨, '2024 오프쇼어 유저 컨퍼런스'서 해양 엔지니어링 기술 혁신 소개",포스코이엔씨가 해양 엔지니어링 분야에서의 경쟁력을 더욱 강화하겠다는 계획을 밝혔다....,2024.07.01,https://kidd.co.kr/news/237311
32,"포스코 E&C, SYNCHRO 활용 '용인 에코타운 BIM 기반 공정 및 원가관리' 발표",포스코 E&C가 벤틀리시스템즈(Bentley Systems) 주최의 '2023 Go...,2023.11.28,https://kidd.co.kr/news/234931
34,LG전자-포스코 ‘AI 자율주행로봇’ 실증 사업 진행,"LG전자가 로봇, 인공지능(AI) 기술 등을 활용해 스마트팩토리 기술 고도화를 실현...",2023.11.15,https://kidd.co.kr/news/234813
59,"다가오는 ESG 공시…포스코, 경영층 의지 바탕으로 대비","한국 기업은 2025년부터 단계적으로 환경‧사회‧지배구조(ESG, Environme...",2022.12.21,https://kidd.co.kr/news/230641
60,"ESG 경영 관심도 1위 철강업계, 포스코그룹 포스코홀딩스",지난 3분기 국내 철강업체 가운데 포스코그룹의 포스코홀딩스가 ESG 경영에 많은 관...,2022.12.14,https://kidd.co.kr/news/230558
64,"화학·첨단소재 기업 ESG 경영 큰 관심, 1위 롯데케미칼 2위 포스코케미칼 順",롯데케미칼이 올 3분기 화학·첨단소재 주요 11개사 중 ESG경영에 가장 큰 관심을...,2022.11.10,https://kidd.co.kr/news/230087
81,"SPX FLOW의 상변화 복합식 에어 드라이어, 포스코 에너지 절감","식음료, 제약 및 일반 산업군에서 공정 솔루션을 제공하는 기업 SPX FLOW, I...",2022.03.23,https://kidd.co.kr/news/226332
